In [1]:
import os

In [ ]:
!pip install langchain

In [ ]:
# !pip install chromadb

!pip install faiss-cpu


In [ ]:
!pip install sentence_transformers #Hagging Face Embedings

In [ ]:
!pip install openllm

In [ ]:
!git clone https://github.com/JagdishKolhe/qa-chat-rag.git

In [ ]:
!pwd

In [8]:
kd_file_path = '/content/qa-chat-rag/content/source_data/KnowledgeDocument.txt'

In [9]:
from langchain.document_loaders import TextLoader

loader = TextLoader(kd_file_path, encoding='utf-8')
docs = loader.load()
markdown_document = ' '.join([d.page_content for d in docs])


In [10]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [11]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits_all = markdown_splitter.split_text(markdown_document)

print(f"Total document chunks: {len(md_header_splits_all)}")

Total document chunks: 25


In [12]:
'''
Although llm is able to handle dirty chars, lets remove all astrics from original content.
We may get very minor improvment in answers from llm
'''
import re
regex_search_term = '\**'
regex_replacement = ''
for doc in md_header_splits_all:
    doc.page_content = re.sub(regex_search_term, regex_replacement, doc.page_content)
    for each in doc.metadata:
        doc.metadata[each] = re.sub(regex_search_term, regex_replacement, doc.metadata[each])



In [13]:
md_header_splits_all[2].page_content

# TODO md_header_splits_all[24] FAQ may needs to split properly.

"In India, two types of PAN cards are available: e-PAN card and physical PAN card.  \n1. e-PAN card: An e-PAN card is a digitally-signed PAN card issued in electronic format. It contains the same PAN details as a physical PAN card but is available in a digital format. It can be downloaded online and used as a valid identification document for various purposes. The e-PAN card is usually issued in a PDF format.\n2. Physical PAN card: A physical PAN card is a laminated card with your PAN details printed on it. It is a physical document that can be carried and used as a valid identification proof. The physical PAN card is sent to the applicant's registered address by post.  \nBoth e-PAN and physical PAN cards have the same validity and can be used for identification purposes. The choice between the two depends on the applicant's preference and requirements."

In [14]:
'''
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

embedings1 = model.encode(md_header_splits[2].page_content)
len(embedings1)
'''

'\nfrom sentence_transformers import SentenceTransformer\nmodel = SentenceTransformer(\'all-MiniLM-L6-v2\')\n\n#Our sentences we like to encode\nsentences = [\'This framework generates embeddings for each input sentence\',\n    \'Sentences are passed as a list of string.\',\n    \'The quick brown fox jumps over the lazy dog.\']\n\n#Sentences are encoded by calling model.encode()\nembeddings = model.encode(sentences)\n\n#Print the embeddings\nfor sentence, embedding in zip(sentences, embeddings):\n    print("Sentence:", sentence)\n    print("Embedding:", embedding)\n    print("")\n\nembedings1 = model.encode(md_header_splits[2].page_content)\nlen(embedings1)\n'

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

text = "This is a test document."

query_result = embeddings.embed_query(text)
print(len(query_result))

doc_result = embeddings.embed_documents([text])
print(len(doc_result))



768
1


In [16]:
'''
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma/'

# !rm -rf ./docs/chroma  # remove old database files if any

vectordb = Chroma.from_documents(
    documents=md_header_splits_all,
    embedding=embeddings,
    persist_directory=persist_directory
)
# vectordb.persist()
print(vectordb._collection.count())
'''

"\nfrom langchain.vectorstores import Chroma\n\npersist_directory = 'docs/chroma/'\n\n# !rm -rf ./docs/chroma  # remove old database files if any\n\nvectordb = Chroma.from_documents(\n    documents=md_header_splits_all,\n    embedding=embeddings,\n    persist_directory=persist_directory\n)\n# vectordb.persist()\nprint(vectordb._collection.count())\n"

In [17]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents=md_header_splits_all,
    embedding=embeddings,
)

In [18]:
question = "What are the documents required to apply for the new pan"

docs = vectordb.similarity_search(question,k=3)
print(f"Number documents {len(docs)}")
for idx, doc in enumerate(docs):
    print(f"\nDocument {idx}\n {doc}")

Number documents 3

Document 0
 page_content='1. If you remember your PAN number:\n- Pan number\n- Passport(Any Country) / OCI Card\n- Passport Size Photograph\n- Overseas address proof with zip code (Supporting documents - Indian NRO/NRE Account statement or Overseas bank statement or Utility bill)\n2. If you don’t remember your PAN number:\nThis gets tricky in most cases so our representative will reach out to you to inform you about the process.  \nDo you remember your PAN card number?' metadata={'Header 1': 'PAN Card Application Process', 'Header 2': 'Reprinting lost Aadhaar Card', 'Header 3': 'Documents required for reprinting the lost PAN card'}

Document 1
 page_content='If you have Aadhaar card  \nNo other document is required. You can get your pan card through your Aadhaar card in 10 minutes.  \nIf you don’t have an Aadhaar card  \n- Passport(Any Country) / OCI Card\n- Passport Size Photograph\n- Overseas address proof with zip code (Supporting documents - Indian NRO/NRE Accou

In [19]:
docs = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3)

print(f"Number documents {len(docs)}")
for idx, doc in enumerate(docs):
    print(f"\nDocument {idx}\n {doc}")

Number documents 2

Document 0
 page_content='1. If you remember your PAN number:\n- Pan number\n- Passport(Any Country) / OCI Card\n- Passport Size Photograph\n- Overseas address proof with zip code (Supporting documents - Indian NRO/NRE Account statement or Overseas bank statement or Utility bill)\n2. If you don’t remember your PAN number:\nThis gets tricky in most cases so our representative will reach out to you to inform you about the process.  \nDo you remember your PAN card number?' metadata={'Header 1': 'PAN Card Application Process', 'Header 2': 'Reprinting lost Aadhaar Card', 'Header 3': 'Documents required for reprinting the lost PAN card'}

Document 1
 page_content='If you have Aadhaar card  \nNo other document is required. You can get your pan card through your Aadhaar card in 10 minutes.  \nIf you don’t have an Aadhaar card  \n- Passport(Any Country) / OCI Card\n- Passport Size Photograph\n- Overseas address proof with zip code (Supporting documents - Indian NRO/NRE Accou

In [20]:
from langchain.llms import OpenLLM

llm = OpenLLM(
    model_name="dolly-v2",
    model_id="databricks/dolly-v2-3b",
    # model_id="databricks/dolly-v2-7b", # Colab gives error for model loading
    # model_id="databricks/dolly-v2-12b", # Colab gives error for model loading
    temperature=0.94,
    repetition_penalty=1.2,
)

In [22]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


In [23]:
result = qa_chain({"query": question})
result

{'query': 'What are the documents required to apply for the new pan',
 'result': 'Firstly if you have PAN card then you only need passport,address proof with zip code,PIO card and OCOI card.\n \nBut if you dont have any of these 4 cards than you should take these 2 as supporting docs while applying for pan.It will help us trace back if you actually live abroad.\n---\nAnd also some banks may ask to provide some ID like company registration documents for verification purpose before getting the bank account opened based on which we\'ll be able to track your bank accounts,if they reside outside india.\nTip: Banks might keep a record of all the information related to foreign residents based on their application for opening of bank accounts \n\nYou can confirm your status via below link \nhttps://aadhaarnri.gov.in/home.html\n\n<|endoftext|>Q:\n\nTheory Explanation "bessel p n" + "w" = 0\n\nIn this page https://en.wikipedia.org/wiki/W_function#Specializations there\'s the formula:\n$$\nJ_{\\b

In [24]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)



In [25]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [28]:
result1 = qa_chain({"query": question})
print(f"Question: {result1['query']}\nAnswer:\n{result1['result']}")

# result1

Question: What are the documents required to apply for the new pan
 Answer:
The specified documents include 
Passport
OCI Card
Other passport size photograph
Overseas address Proof with Zip Code supporting documents —Indian NRo/NRoA account statementor Overseasbank Statement orUtility Bill


In [37]:
def ask_qn(question) :
  result1 = qa_chain({"query": question})
  print(f"\nQuestion: {result1['query']}\nAnswer:\n{result1['result']}\n----------------------------\n")


In [39]:
ask_qn("What are the documents required to apply for the new pan")


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Question: What are the documents required to apply for the new pan
Answer:
For RBI permission or endorsement, the applicant shall attach all the photocopies of below documents along with Application Form 60 which he has to fill on line ( https://www.rbinetex.gov.in/pdffiles/PressRelease_PANApplicationForm60.pdf ). Also printout the permission slip and keep it alongside with application form. 
RBI reference : ITDB53808190012356924457425961552685872381 and ITDB53808217002449011475484746042532341811899
Along with above two attachments also need to send First Page and Last Page of utility bill with due date.
Additionally, if applicant wants further guidance to do online Payment then visit www.itdbindia.org website fill in the details like Customer ID and Password, choose mode of Payment and click continue. Following step will redirect you back to Online Payment and you will get redirected again after submission. The only difference would be no any acknowledgement message from IRS. This co

In [ ]:
ask_qn("Process to change the citizenship on PAN Card")


In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce",
)

In [ ]:
result2 = qa_chain_mr({"query": question})
result2

Token indices sequence length is longer than the specified maximum sequence length for this model (2331 > 1024). Running this sequence through the model will result in indexing errors


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 result2 = qa_chain_mr({"query": question})                                                   │
│   2 result2                                                                                      │
│   3                                                                                              │
│                                                                                                  │
│ C:\Users\jkolhe\Miniconda3\envs\qa\lib\site-packages\langchain\chains\base.py:243 in __call__    │
│                                                                                                  │
│   240 │   │   │   )                                                                              │
│   241 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   242 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 243 │   │   │   raise e                                                                        │
│   244 │   │   run_manager.on_chain_end(outputs)                                                  │
│   245 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   246 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ C:\Users\jkolhe\Miniconda3\envs\qa\lib\site-packages\langchain\chains\base.py:237 in __call__    │
│                                                                                                  │
│   234 │   │   )                                                                                  │
│   235 │   │   try:                                                                               │
│   236 │   │   │   outputs = (                                                                    │
│ ❱ 237 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   238 │   │   │   │   if new_arg_supported                                                       │
│   239 │   │   │   │   else self._call(inputs)                                                    │
│   240 │   │   │   )                                                                              │
│                                                                                                  │
│ C:\Users\jkolhe\Miniconda3\envs\qa\lib\site-packages\langchain\chains\retrieval_qa\base.py:131   │
│ in _call                                                                                         │
│                                                                                                  │
│   128 │   │   │   docs = self._get_docs(question, run_manager=_run_manager)                      │
│   129 │   │   else:                                                                              │
│   130 │   │   │   docs = self._get_docs(question)  # type: ignore[call-arg]                      │
│ ❱ 131 │   │   answer = self.combine_documents_chain.run(                                         │
│   132 │   │   │   input_documents=docs, question=question, callbacks=_run_manager.get_child()    │
│   133 │   │   )                                                                                  │
│   134                                                                                            │
│                                                                                                  │
│ C:\Users\jkolhe\Miniconda3\envs\qa\lib\site-packages\langchain\chains\base.py:445 in run         │
│                                                            

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type="stuff",
)

In [ ]:
result3 = qa_chain_mr({"query": question})
result3

In [ ]:
i